# Importações

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy as sa

# Criando base de Dados

In [2]:
db = create_engine('sqlite:///db_roos.sqlite', echo=False)
conn = db.connect()

# Carregando os Datasets

In [ ]:
#dataset
pessoas = pd.read_csv( '/mnt/d/repos/ROOS/data/PESSOAS.csv', sep = ';' )

schema_pessoas ="""
    CREATE TABLE PESSOAS(
        IDPESSOA           INTEGER,
        NOME               TEXT
)
"""
#create schema
conn.execute( schema_pessoas )

# insert data into table
pessoas.to_sql( 'PESSOAS', con=conn, if_exists='append', index=False)

In [4]:
query = """
    SELECT *
    FROM pessoas
"""

table = pd.read_sql_query(query, conn)
table

,IDPESSOA,NOME
0,25,DIEGO
1,26,GABRIEL
2,27,BRUNA
3,28,ALEXSANDRO


In [5]:
#dataset
romaneio = pd.read_csv( '/mnt/d/repos/ROOS/data/ROMANEIO.csv', sep = ';' )

schema_romaneio ="""
    CREATE TABLE ROMANEIO(
        ROMANEIO           INTEGER,
        IDPESSOA           INTEGER,
        DATA                  TEXT,
        ITEM               INTEGER,
        PESO               INTEGER
)
"""
#create schema
conn.execute( schema_romaneio )

# insert data into table
romaneio.to_sql( 'ROMANEIO', con=conn, if_exists='append', index=False)

6

In [6]:
query = """
    SELECT *
    FROM romaneio
"""

table = pd.read_sql_query(query, conn)
table

,ROMANEIO,IDPESSOA,DATA,ITEM,PESO
0,1,25,03/02/2023,1001,3500
1,2,26,04/02/2023,1001,5000
2,3,27,05/02/2023,1002,10000
3,4,28,06/02/2023,1001,8000
4,5,25,04/01/2023,1001,8000
5,1,25,04/01/2023,1001,5000


In [7]:
#dataset
romaneio_teste = pd.read_csv( '/mnt/d/repos/ROOS/data/ROMANEIO_TESTE.csv', sep = ';' )

schema_romaneio_teste ="""
    CREATE TABLE ROMANEIO_TESTE(
        ROMANEIO           INTEGER,
        IDTESTE            INTEGER,
        VALOR               OBJECT
)
"""
#create schema
conn.execute( schema_romaneio_teste )

# insert data into table
romaneio_teste.to_sql( 'ROMANEIO_TESTE', con=conn, if_exists='append', index=False)

5

In [8]:
query = """
    SELECT *
    FROM romaneio_teste
"""

table = pd.read_sql_query(query, conn)
table

,ROMANEIO,IDTESTE,VALOR
0,1,1,"17,5"
1,1,2,15
2,2,2,14
3,2,1,17
4,4,3,15


In [9]:
#dataset
teste = pd.read_csv( '/mnt/d/repos/ROOS/data/TESTE.csv', sep = ';' )

schema_teste ="""
    CREATE TABLE TESTE(
        IDTESTE           INTEGER,
        NOME                 TEXT
)
"""
#create schema
conn.execute( schema_teste )

# insert data into table
teste.to_sql( 'TESTE', con=conn, if_exists='append', index=False)

3

In [10]:
query = """
    SELECT *
    FROM teste
"""

table = pd.read_sql_query(query, conn)
table

,IDTESTE,NOME
0,1,UMIDADE
1,2,IMPUREZA
2,3,OUTROS


# Teste

In [11]:
query = """
    SELECT
        r.romaneio,
        p.nome AS NOME_PESSOA,
        MAX (CASE WHEN t.nome = "UMIDADE" THEN rt.valor END) AS UMIDADE,
        MAX (CASE WHEN t.nome = "IMPUREZA" THEN rt.valor END) AS IMPUREZA,
        MAX (CASE WHEN t.nome = "OUTROS" THEN rt.valor END) AS OUTROS,
        r.peso AS PESOTOTAL_ROMANEIO
        

    
    FROM romaneio r LEFT JOIN pessoas p ON r.idpessoa = p.idpessoa
                    LEFT JOIN romaneio_teste rt ON r.romaneio = rt.romaneio
                    LEFT JOIN teste t ON rt.idteste = t.idteste
                    
    GROUP BY r.romaneio           

  """

table = pd.read_sql_query(query, conn)
table

,ROMANEIO,NOME_PESSOA,UMIDADE,IMPUREZA,OUTROS,PESOTOTAL_ROMANEIO
0,1,DIEGO,"17,5",15.0,NaN,5000
1,2,GABRIEL,17,14.0,NaN,5000
2,3,BRUNA,None,NaN,NaN,10000
3,4,ALEXSANDRO,None,NaN,15.0,8000
4,5,DIEGO,None,NaN,NaN,8000


In [ ]:
CREATE FUNCTION SALDO_PESSOA(@ITEM,@DATA,@PESSOA)
RETURNS TABLE
AS
RETURN (SELECT PESO
        FROM  ROMANEIO
        WHERE DATA == @DATA
        OR ITEM == @ITEM
        OR IDPESSOA == @PESSOA)